## Applied Data Science Capstone

This notebook is intended to collect all relevant code linked to tasks for the IBM "Applied Data Science Capstone" certification

In [4]:
import pandas as pd
import numpy as np